<a href="https://colab.research.google.com/github/kjfms/NIA-Project/blob/main/%EC%9E%A5%ED%9D%A5%EA%B5%B0%EC%B2%AD_%EC%A3%BC%EC%9A%94%EB%AF%BC%EC%9B%90%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 주요민원분석

### 패키지 설치

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!apt-get update !apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy
!JAVA_HOME="C:\Program Files\Java\jdk-16.0.2"

In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from collections import Counter #카운트
from konlpy.utils import pprint
from konlpy.tag import Mecab

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 빈도 수 TOP10 추출 

### 파일 불러오기
#### '청구제목' '청구내용' '공개내용' 안에 있는 명사들(100회 이상 반복된 것만) 추출 후 그 명사들을 포함하는 행 수 추출 후 top10 추출

In [ ]:
c = open('/content/drive/MyDrive/test/3column.txt', 'r', encoding='CP949')
data3 = c.read()

In [ ]:
okt = Okt()
noun = okt.nouns(data3)

In [ ]:
# 한글자 명사 제거
for i,v in enumerate(noun):
        if len(v)<2:
            noun.pop(i)

In [ ]:
# 100번이상 반복된 것만 추출
count = Counter(noun)
noun_dict = {k : v for k, v in  count.items() if v >= 100}

### 불용어처리

In [ ]:
dd = open('/content/drive/MyDrive/test/불용어사전_논문.txt', 'r', encoding='utf-8')
dd = dd.read()

In [ ]:
result = {}
for key,value in noun_dict.items():
  if key not in dd:
    result[key] = value



---



---



In [ ]:
# csv로 저장
import csv
with open('3column_result.csv','w',encoding = 'utf-8-sig') as f:
    w = csv.writer(f)
    w.writerow(result.keys())
    w.writerow(result.values())

### 특정 단어 포함 행 수 추출
#### 단어의 빈도가 아닌 민원의 빈도를 추출하기 위해 특정 단어가 포함된 행을 추출(이때 중복된 행은 제거)

In [ ]:
# 위에서 저장한 csv에서 키워드만 추출

kw3_list = pd.read_csv('/content/drive/MyDrive/test/3column_keyword.csv',header = None, encoding = 'utf-8')

In [ ]:
kw_list3 = kw3_list.values.tolist() #df -> 2차원 list
kw3 = sum(kw_list3, []) # 2차원 list -> 1차원 list

In [ ]:
# 원본데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/test/청구서전체현황(2020-01-01_2020-12-31).csv',encoding = 'utf-8')

In [ ]:
# 원본데이터에서 각 키워드가 포함된 행 수 추출

a = []

for d in kw3:
    data1 = data[data['청구제목'].str.contains(d, na = False)]
    data2 = data[data['청구내용'].str.contains(d, na = False)]
    data3 = data[data['공개내용'].str.contains(d, na = False)]
    result = pd.concat([data1,data2,data3])
    result.drop_duplicates(inplace=True)
    a.append(len(result))

In [ ]:
c = pd.DataFrame(a) # list to df
q = pd.concat([kw3_list,c], axis=1) # 키워드 리스트 열과 그 키워드가 포함된 행 수 열 병합
q.columns = ['단어', '빈도'] # 열명 설정

print(q)

In [ ]:
# csv로 저장
q.to_csv("hi3.csv", encoding = 'utf-8-sig')



---



---



In [ ]:
# 수동으로 불용어 다시 제거(-83)하고 csv파일로 불러옴

In [ ]:
result3 = pd.read_csv('/content/drive/MyDrive/test/3번과제 - 빈도 수 집계 최종 결과.csv', header=None, encoding = 'utf-8')

### 워드클라우드

In [ ]:
result5 = result3.set_index(0).to_dict() # 첫번째 열을 인덱스로 설정후 df -> dict 변환

In [ ]:
# 이렇게 가져오면 2중 dict이 되어서 그 안에 있는 dict만 추출
result6 = result5[1]

In [ ]:
# 폰트 설정
font = '/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf'

# 워드클라우드 설정
wc = WordCloud(font_path=font,\
		background_color="white", \
		width=2000, \
		height=1000, \
		max_words=100, \
		max_font_size=300)
wc = wc.generate_from_frequencies(result6)

plt.figure(figsize=(10,10))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

## TF-IDF 점수 추출

In [ ]:
testdata = data['청구내용'].values.tolist()

In [ ]:
testdata[:3]

In [ ]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                   lowercase=False,
                                   tokenizer=None,
                                   preprocessor=None,
                                   min_df=5,
                                   smooth_idf=True)

In [ ]:
tfidf_vector = tfidf_vectorizer.fit_transform(testdata)

In [ ]:
tfidf_scores = tfidf_vector.toarray().sum(axis=0)

In [ ]:
tfidf_idx = np.argsort(-tfidf_scores)

In [ ]:
tfidf_scores

In [ ]:
god = pd.DataFrame(tfidf_vector.toarray(), columns=tfidf_vectorizer.get_feature_names())  

In [ ]:
god.to_csv('sample2.csv', encoding='euc-kr')



---



---



In [ ]:
tfidf_vectorizer.fit(testdata) # 단어를 학습시킴 
tfidf_vectorizer.vocabulary_ # 단어사전을 출력 
sorted(tfidf_vectorizer.vocabulary_.items()) # 단어사전 정렬

In [ ]:
 tfidf_vectorizer.idf_ #idf 벡터화

In [ ]:
a = tfidf_vectorizer.transform(testdata).toarray()

In [ ]:
df = pd.DataFrame(a)
df.to_csv('sample.csv')

In [ ]:
tf = pd.DataFrame(tfidf_vectorizer.transform(testdata).toarray(), columns=tfidf_vectorizer.get_feature_names())  

In [ ]:
tf.to_csv('sample1.csv', encoding='euc-kr')



---



---



In [ ]:
from math import log

In [ ]:
N = len(testdata) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in testdata:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

## TF-IDF 점수 TOP10 추출

### 파일 불러오기

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/test/청구서전체현황(2020-01-01_2020-12-31).csv', encoding = 'utf-8')

In [ ]:
data = data[["접수일","청구제목","청구내용","공개내용"]]

In [ ]:
tfidf_top10 = ['보건소','상금','토양오염','음식점','식품','폐기물','의약품','주택','건축','정화']

### TOP10 단어 포함 행 수 추출

In [ ]:
# 각 키워드가 포함된 행 수 추출

a = []

for d in tfidf_top10:
    data1 = data[data['청구제목'].str.contains(d, na = False)]
    data2 = data[data['청구내용'].str.contains(d, na = False)]
    data3 = data[data['공개내용'].str.contains(d, na = False)]
    result = pd.concat([data1,data2,data3])
    a.append(len(result))
    result.drop_duplicates(inplace=True)
    print(result)

In [ ]:
b = pd.DataFrame(a) # list to df
print(b)



---



---



In [ ]:
# csv로 저장
b.to_csv("plz.csv", encoding = 'utf-8-sig')

### TOP10 민원 키워드 현황 및 발생 추이 

In [ ]:
# 필요한 패키지와 라이브러리를 가져옴
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker # 그래프 축 간격

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/test/청구서전체현황(2020-01-01_2020-12-31).csv',encoding = 'utf-8')
data = data[['접수일', '청구제목', '청구내용', '공개내용']] # 월별로 묶기 위해 '접수일' 열 추가

In [ ]:
import datetime

data['접수일'] = pd.to_datetime(data['접수일']) # '접수일'열 datetime 형식으로 변경
data['접수월'] = data['접수일'].dt.month # '접수일'에서 월만 추출해서 '접수월'열 생성

In [ ]:
tfidf_top10 = ['보건소','상금','토양오염','음식점','식품','폐기물','의약품','주택','건축','정화'] # tf-idf 점수 TOP10 키워드

In [ ]:
# TOP10 키워드가 들어간 행 추출하여 df로 병합
# 월별로 그룹화

df_base= pd.DataFrame(columns=['월','민원수'])

for b in tfidf_top10:
    data1 = data[data['청구제목'].str.contains(b, na = False)]
    data2 = data[data['청구내용'].str.contains(b, na = False)]
    data3 = data[data['공개내용'].str.contains(b, na = False)]
    ldata = pd.concat([data1,data2,data3])
    ldata.drop_duplicates(inplace=True)

    group_month = ldata.groupby(ldata['접수월'], as_index=False)['접수일'].count() # 월 별로 민원 발생 빈도 추출 (접수일에는 NULL 없음)
    month_data = pd.DataFrame(group_month) # df로 변환
    month_data.columns = ['월', '민원수'] # 열명 변경
    df_base=df_base.append(month_data)
df_base

In [ ]:
# csv로 저장
df_base.to_csv("plz2.csv", encoding = 'utf-8-sig')

In [ ]:
# TOP10 키워드가 들어간 행 추출하여 df로 병합
# 월별로 그룹화하고 시각화

for b in tfidf_top10:
    data1 = data[data['청구제목'].str.contains(b, na = False)]
    data2 = data[data['청구내용'].str.contains(b, na = False)]
    data3 = data[data['공개내용'].str.contains(b, na = False)]

    ldata = pd.concat([data1,data2,data3])
    ldata.drop_duplicates(inplace=True)

    group_month = ldata.groupby(ldata['접수월'], as_index=False)['접수일'].count() # 월 별로 민원 발생 빈도 추출 (접수일에는 NULL 없음)
    month_data = pd.DataFrame(group_month) # df로 변환
    month_data.columns = ['월', '민원수'] # 열명 변경

    plt.figure(figsize=(6,4))
    plt.plot(month_data['월'], month_data['민원수'], color = 'blue')
    plt.xlim(1,12)                                                   # x축 범위
    ax=plt.axes()
    ax.xaxis.set_major_locator(ticker.MultipleLocator(2))            # x축 간격
    plt.xlabel('월')
    plt.ylabel('민원수')
    plt.title(b, fontsize = 20)
    plt.rc('font', family='NanumBarunGothic') 
    plt.show()